# Feature Extraction from Text

In [ ]:
# Imports

import numpy as np
import pandas as pd

## Load a dataset

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/renatomaaliw3/public_files/refs/heads/master/Data%20Sets/NLP/smsspamcollection.tsv', sep = '\t')
df

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,160,8
5568,ham,Will ü b going to esplanade fr home?,36,1
5569,ham,"Pity, * was in mood for that. So...any other s...",57,7
5570,ham,The guy did some bitching but I acted like i'd...,125,1


In [ ]:
# Check Missing Values

df.isnull().sum()

,0
label,0
message,0
length,0
punct,0


In [ ]:
# ham vs spam count

df['label'].value_counts()

,count
label,
ham,4825
spam,747


## Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X = df['message']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

## Counter Vectorization

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

In [ ]:
# If you look a X
X

,message
0,"Go until jurong point, crazy.. Available only ..."
1,Ok lar... Joking wif u oni...
2,Free entry in 2 a wkly comp to win FA Cup fina...
3,U dun say so early hor... U c already then say...
4,"Nah I don't think he goes to usf, he lives aro..."
...,...
5567,This is the 2nd time we have tried 2 contact u...
5568,Will ü b going to esplanade fr home?
5569,"Pity, * was in mood for that. So...any other s..."
5570,The guy did some bitching but I acted like i'd...


In [ ]:
# Fit Vectorize to the Data (build a vocabulary, count the number of words...)

count_vect.fit(X_train)
X_train_counts = count_vect.transform(X_train)

In [ ]:
# Transform the original text message to vector (alternate)

X_train_counts = count_vect.fit_transform(X_train)
X_train_counts

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 49992 stored elements and shape (3733, 7082)>

In [ ]:
# Note: Across 3733 messages, there are 7082 unique words

## TfidfTransformer

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 49992 stored elements and shape (3733, 7082)>

## TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train)
X_train_tfidf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 49992 stored elements and shape (3733, 7082)>

# Train a Classifier

In [ ]:
from sklearn.svm import LinearSVC

clf = LinearSVC()
clf.fit(X_train_tfidf, y_train)

LinearSVC()

In [ ]:
# Remember we only vectorized the train data, how about the test data? We should repeat the process for the test data?

## Pipeline (Makes life easier)

In [ ]:
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC())])

text_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [ ]:
# Predictions

predictions = text_clf.predict(X_test)
X_test

,message
3245,Squeeeeeze!! This is christmas hug.. If u lik ...
944,And also I've sorta blown him off a couple tim...
1044,Mmm thats better now i got a roast down me! i...
2484,Mm have some kanji dont eat anything heavy ok
812,So there's a ring that comes with the guys cos...
...,...
4944,Check mail.i have mailed varma and kept copy t...
3313,I know you are serving. I mean what are you do...
3652,Want to send me a virtual hug?... I need one
14,I HAVE A DATE ON SUNDAY WITH WILL!!


In [ ]:
# Performance

from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Confusion Matrix

print(confusion_matrix(y_test, predictions))

[[1586    7]
 [  12  234]]


In [ ]:
# Classification Report

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [ ]:
# Accuracy Scores (Single Value)

from sklearn import metrics
metrics.accuracy_score(y_test, predictions)

0.989668297988037

In [ ]:
# Prediction on Unseen Data

text_clf.predict(['Congratulations! You are a winner. TEXT WON to 4425', 'Dear Renato, you are invited to attend the Faculty Meeting tomorrow at 9 AM at the Dean\'s Office'])

array(['spam', 'ham'], dtype=object)